In [1]:
from experiments.experiment_config import (
    ALL_LIGHTNING_MODELS,
    DRIVE_AND_ACT_VIEWPOINTS,
    get_drive_and_act_loaders,
    driver_and_act_pretrained_map
)
from modules.lifter_2d_3d.utils.evaluation import Evaluator
viewpoint = list(driver_and_act_pretrained_map.keys())[0]
constructed_loader = get_drive_and_act_loaders(viewpoint)

pretrained_model_path = 'saved_lifter_2d_3d_model/rq2/LitSimpleBaselineLinear/synthetic_cabin_ir_1m/all_views'

with open(f'{pretrained_model_path}/best_model_path.txt', 'r') as f:
    pretrained_checkpoint_path = f.readline()
lit_model = ALL_LIGHTNING_MODELS[0].load_from_checkpoint(pretrained_checkpoint_path)
lit_model.set_all_activities(constructed_loader['all_activities'])
lit_model.eval();

In [2]:
import torch
import numpy as np
batch = next(iter(constructed_loader['train_loader']))
x, y, valid, activities = lit_model.preprocess_input(*lit_model.preprocess_batch(batch))
y_hat = lit_model.forward(x.to('cuda'))

torch.Size([64, 13, 2])
torch.Size([64, 26])


In [3]:
evaluator = Evaluator(is_procrustes=False)
p_evaluator = Evaluator(is_procrustes=True)

In [4]:
evaluator.add_result(
    y_hat.detach().cpu().numpy(),
    y.detach().cpu().numpy(),
    valid.detach().cpu().numpy(),
    activities
)

In [5]:
evaluator.get_result()

([{'nose': 107.0462646484375,
   'left_eye': 103.03743743896484,
   'right_eye': 104.2566146850586,
   'left_ear': 21.95159339904785,
   'right_ear': 55.921180725097656,
   'left_shoulder': 16.93596649169922,
   'right_shoulder': 16.93478775024414,
   'left_elbow': 116.03675842285156,
   'right_elbow': 122.07261657714844,
   'left_wrist': 165.49891662597656,
   'right_wrist': 187.376953125,
   'left_hip': 90.14336395263672,
   'right_hip': 94.05154418945312}],
 108.201764523983,
 None,
 None)

In [6]:
p_evaluator.add_result(
    y_hat.detach().cpu().numpy(),
    y.detach().cpu().numpy(),
    valid.detach().cpu().numpy(),
    activities
)

In [7]:
p_evaluator.get_result()

([{'nose': 49.63608970865607,
   'left_eye': 46.93726655386854,
   'right_eye': 47.2366017929744,
   'left_ear': 12.436410645022988,
   'right_ear': 33.14387112914119,
   'left_shoulder': 52.507817730656825,
   'right_shoulder': 50.08431685564574,
   'left_elbow': 63.95507730485406,
   'right_elbow': 71.94398567662574,
   'left_wrist': 101.92400842788629,
   'right_wrist': 107.12514616898261,
   'left_hip': 86.76080190343782,
   'right_hip': 96.67466260725632}],
 73.38394161784642,
 None,
 None)